In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

BASE_DIR       = '../'
IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 32
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 16
LEARNING_RATE  = 0.001

In [3]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [5]:
X_train = np.stack([X_train_xy[:, :, :, 0], X_train_yz[:, :, :, 0], X_train_zx[:, :, :, 0]], axis=3)
X_test = np.stack([X_test_xy[:, :, :, 0], X_test_yz[:, :, :, 0], X_test_zx[:, :, :, 0]], axis=3)
y_train = y_train_xy
y_test = y_test_xy

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(32, 32, 3), activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='softmax'))

In [7]:
opt = tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['accuracy']
)
history = model.fit(X_train, y_train, epochs=EPOCHS)

Epoch 1/16
30/30 [==============================] - 3s 72ms/step - loss: 246.0351 - accuracy: 0.0833
Epoch 2/16
30/30 [==============================] - 2s 68ms/step - loss: 2.7611 - accuracy: 0.1180
Epoch 3/16
30/30 [==============================] - 2s 67ms/step - loss: 2.6643 - accuracy: 0.1587
Epoch 4/16
30/30 [==============================] - 2s 70ms/step - loss: 2.5633 - accuracy: 0.1857
Epoch 5/16
30/30 [==============================] - 2s 68ms/step - loss: 2.2426 - accuracy: 0.3207
Epoch 6/16
30/30 [==============================] - 2s 68ms/step - loss: 1.8495 - accuracy: 0.4650
Epoch 7/16
30/30 [==============================] - 2s 69ms/step - loss: 1.3601 - accuracy: 0.5940
Epoch 8/16
30/30 [==============================] - 2s 68ms/step - loss: 1.1684 - accuracy: 0.6580
Epoch 9/16
30/30 [==============================] - 2s 73ms/step - loss: 0.7578 - accuracy: 0.7799
Epoch 10/16
30/30 [==============================] - 2s 69ms/step - loss: 0.5634 - accuracy: 0.8429
Epoch 1

In [8]:
y_pred_hot = model.predict(X_test)
y_pred = np.argmax(y_pred_hot, axis=1)
print(classification_report(y_test.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.80      0.80        10
         1.0       0.58      0.70      0.64        10
         2.0       0.67      0.20      0.31        10
         3.0       0.60      0.60      0.60        10
         4.0       0.78      0.70      0.74        10
         5.0       0.67      0.40      0.50        10
         6.0       1.00      0.40      0.57        10
         7.0       0.60      0.60      0.60        10
         8.0       0.48      1.00      0.65        10
         9.0       0.56      0.50      0.53        10
        10.0       0.53      1.00      0.69        10
        11.0       0.17      0.20      0.18        10
        12.0       0.00      0.00      0.00        10
        13.0       0.12      0.10      0.11        10
        14.0       0.00      0.00      0.00        10
        15.0       0.36      0.50      0.42        10

    accuracy                           0.48       160
   macro avg       0.49   